In [42]:
### importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.utils import shuffle
import os

In [2]:
### loading csv files 
test_path="../input/cassava-leaf-disease-classification/test_images"
train_path="../input/cassava-leaf-disease-classification/train_images"
label_path="../input/cassava-leaf-disease-classification/train.csv"
names_path="../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
train_data=pd.read_csv(label_path)

In [3]:
### Basic Statistics 
print("Number of Rows in the Train dataset  :",train_data.shape[0])
train_data.head()

Number of Rows in the Train dataset  : 21397


,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [4]:
### Label Value counts 
train_data.label.value_counts()
train_data.label=train_data.label.map(str)

In [5]:
### splitting the Train dataset in to training and Validation
train_df,val_df,_,_=train_test_split(train_data,train_data.label,stratify=train_data.label,test_size=0.1)
print("Shape of the Training set :",train_df.shape)
print("Shape of the Validation set :",val_df.shape)

Shape of the Training set : (19257, 2)
Shape of the Validation set : (2140, 2)


In [6]:
train_data.label.value_counts()

3    13158
4     2577
2     2386
1     2189
0     1087
Name: label, dtype: int64

### Experiment 1: 

-> Using Whole dataset at once

-> Deal with the Imabalancing the dataset 

                -> we will use the oversampling on minority classes
                
-> use Inception Model

In [7]:
### we need to oversampling for the minority classes
n_1=len(train_df.loc[train_df.label=="1"])
n_2=len(train_df.loc[train_df.label=="2"])
n_4=len(train_df.loc[train_df.label=="4"])
n_0=len(train_df.loc[train_df.label=="0"])
temp_1=train_df.loc[train_df.label=="1"].sample(n=n_1*4,replace=True)
temp_2=train_df.loc[train_df.label=="2"].sample(n=n_2*4,replace=True)
temp_4=train_df.loc[train_df.label=="4"].sample(n=n_4*4,replace=True)
temp_0=train_df.loc[train_df.label=="0"].sample(n=n_0*4,replace=True)

In [8]:
### concatenate all the Classes
data=shuffle(pd.concat([temp_1,temp_2,temp_4,temp_0,train_df.loc[train_df.label=="3"]],ignore_index=True)).reset_index(drop=True)

In [9]:
data.head()

,image_id,label
0,2247618278.jpg,4
1,2349124978.jpg,2
2,684911390.jpg,2
3,2293657925.jpg,1
4,2437151204.jpg,3


### Importing the Necccessary Libraries for the model

In [44]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Flatten
from keras.models import Sequential
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam,RMSprop
from keras.utils import plot_model
import tensorflow as tf
from keras.models import load_model

### Creating the Data Generator

In [24]:
### configuration for a model
batch=15
img_size=338


In [25]:
### Preparing the Data Generator 
gen_tr=ImageDataGenerator(
                        
                        rotation_range=70,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        #brightness_range=[0.1,0.7],
                        shear_range=0.1,
                        horizontal_flip=True,
                        vertical_flip=True,
                        
                       )

train_gen=gen_tr.flow_from_dataframe(
                                        train_df,
                                        directory=train_path,
                                        x_col="image_id",
                                        y_col="label",
                                        target_size=(img_size,img_size),
                                        class_mode="categorical",
                                        batch_size=batch,
                                        shuffle=True,
                                        interpolation="nearest",
                                        validate_filenames=True
                                    )

gen_val=ImageDataGenerator(
                            
                           )

val_gen=gen_val.flow_from_dataframe(
                                        val_df,
                                        directory=train_path,
                                        x_col="image_id",
                                        y_col="label",
                                        target_size=(img_size, img_size),
                                        class_mode="categorical",
                                        batch_size=batch,
                                        shuffle=True,
                                        interpolation="nearest",
                                        validate_filenames=True
                                    )


Found 19257 validated image filenames belonging to 5 classes.
Found 2140 validated image filenames belonging to 5 classes.


### Building the Model
- here we using the Inception_v3

In [ ]:
base=InceptionV3(
                include_top=False,
                input_tensor=None,
                input_shape=(224,224,3)
                )
for i in range(100):
    base.layers[i].trainable=False

model=Sequential()
model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(5,activation="softmax"))
model.summary()

In [ ]:
### Defining the Parameters
opt=Adam(learning_rate=0.001)
cat_loss=CategoricalCrossentropy(from_logits=False, label_smoothing=0)
## defining the call backs
early_stopping=tf.keras.callbacks.EarlyStopping(
                                                monitor="val_loss",
                                                patience=3,
                                                mode="min",
                                                restore_best_weights=True
                                              )
### Now reducing the learning rate when the model is not improvinig 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
                                                monitor="val_loss",
                                                factor=0.2,
                                                patience=3,
                                                verbose=1,
                                                mode="auto"
                                            )

my_callbacks=[early_stopping,reduce_lr]

In [ ]:
## Compiling the Models
model.compile(loss=cat_loss,optimizer=opt,metrics=["categorical_accuracy"])
### fitting the model
#model.fit(train_gen,epochs=4,validation_data=val_gen,callbacks=my_callbacks)

### Experiment :2 --> EfficientNetB3

In [26]:
base=tf.keras.applications.EfficientNetB3(
                                            include_top=False,
                                            input_tensor=None,
                                            input_shape=(img_size,img_size,3)
                                            )
#index=base.layers.index(base.get_layer("block4a_expand_conv"))
#for i in range(index):
#    base.layers[i].trainable=False

model_EN=Sequential()
model_EN.add(base)
model_EN.add(GlobalAveragePooling2D())
model_EN.add(Flatten())
model_EN.add(Dense(5,activation="softmax"))
model_EN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, 11, 11, 1536)      10783535  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1536)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 7685      
Total params: 10,791,220
Trainable params: 10,703,917
Non-trainable params: 87,303
_________________________________________________________________


In [27]:
### Defining the Parameters
opt=Adam(learning_rate=0.0004)
cat_loss=CategoricalCrossentropy(from_logits=False, label_smoothing=0.2)
## defining the call backs
early_stopping=tf.keras.callbacks.EarlyStopping(
                                                monitor="val_loss",
                                                patience=3,
                                                mode="min",
                                                restore_best_weights=True
                                              )
### Now reducing the learning rate when the model is not improvinig 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
                                                monitor="val_loss",
                                                factor=0.2,
                                                patience=2,
                                                verbose=1,
                                                mode="auto"
                                            )

my_callbacks=[early_stopping,reduce_lr]

num_epochs=15

In [28]:
## Compiling the Models
#model_EN.compile(loss=cat_loss,optimizer=opt,metrics=["categorical_accuracy"])
#model_EN.load_weights("../input/cassava-saved-models/model_efb3_wts.h5")

In [29]:

### fitting the model
#model_EN.fit(train_gen,epochs=num_epochs,validation_data=val_gen,callbacks=my_callbacks)

Epoch 1/15
1284/1284 [==============================] - 863s 662ms/step - loss: 0.9157 - categorical_accuracy: 0.8471 - val_loss: 0.8853 - val_categorical_accuracy: 0.8678
Epoch 2/15
 153/1284 [==>...........................] - ETA: 12:09 - loss: 0.8805 - categorical_accuracy: 0.8610

KeyboardInterrupt: 

In [22]:
#model_EN.save("model_efb3.h5")
#model_EN.save_weights('model_efb3_wts.h5')

In [30]:
base=tf.keras.applications.EfficientNetB4(
                                            include_top=False,
                                            input_tensor=None,
                                            input_shape=(img_size,img_size,3)
                                            )
#index=base.layers.index(base.get_layer("block4a_expand_conv"))
#for i in range(index):
#    base.layers[i].trainable=False

model_EN1=Sequential()
model_EN1.add(base)
model_EN1.add(GlobalAveragePooling2D())
model_EN1.add(Flatten())
model_EN1.add(Dense(5,activation="softmax"))
model_EN1.summary()

71688192/71686520 [==============================] - 2s 0us/step
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb4 (Functional)  (None, 11, 11, 1792)      17673823  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1792)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 8965      
Total params: 17,682,788
Trainable params: 17,557,581
Non-trainable params: 125,207
_________________________________________________________________


In [31]:
### Defining the Parameters
opt=Adam(learning_rate=0.003)
cat_loss=CategoricalCrossentropy(from_logits=False, label_smoothing=0.02)
## defining the call backs
early_stopping=tf.keras.callbacks.EarlyStopping(
                                                monitor="val_loss",
                                                patience=3,
                                                mode="min",
                                                restore_best_weights=True
                                              )
### Now reducing the learning rate when the model is not improvinig 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
                                                monitor="val_loss",
                                                factor=0.2,
                                                patience=2,
                                                verbose=1,
                                                mode="auto"
                                            )

my_callbacks=[early_stopping,reduce_lr]

num_epochs=10

In [36]:
model_EN1.compile(loss=cat_loss,optimizer=opt,metrics=["categorical_accuracy"])
### fitting the model
#model_EN1.fit(train_gen,epochs=num_epochs,validation_data=val_gen,callbacks=my_callbacks)

Epoch 1/10
1284/1284 [==============================] - 911s 710ms/step - loss: 0.4628 - categorical_accuracy: 0.8663 - val_loss: 0.4664 - val_categorical_accuracy: 0.8636
Epoch 2/10
1284/1284 [==============================] - 913s 711ms/step - loss: 0.4467 - categorical_accuracy: 0.8703 - val_loss: 0.4798 - val_categorical_accuracy: 0.8640
Epoch 3/10
1284/1284 [==============================] - 914s 712ms/step - loss: 0.4335 - categorical_accuracy: 0.8753 - val_loss: 0.4927 - val_categorical_accuracy: 0.8551

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00012000000569969416.
Epoch 4/10
1284/1284 [==============================] - 918s 715ms/step - loss: 0.4222 - categorical_accuracy: 0.8811 - val_loss: 0.4601 - val_categorical_accuracy: 0.8701
Epoch 5/10
1284/1284 [==============================] - 913s 711ms/step - loss: 0.4162 - categorical_accuracy: 0.8827 - val_loss: 0.4467 - val_categorical_accuracy: 0.8785
Epoch 6/10
1284/1284 [==============================] - 92

In [37]:
#model_EN1.save("model_efb4.h5")
#model_EN1.save_weights('model_efb4_wts.h5')

### InFerence the Model

In [43]:
import cv2

In [45]:
infer_model=load_model("../input/cassava-saved-models/model_efb4.h5")

In [51]:
img_names=os.listdir(test_path)
predicts=[]
ids=[]
for i in img_names:
    ids.append(i)
    path=os.path.join(test_path,i)
    img=plt.imread(path)
    crop_img=cv2.resize(img,(img_size,img_size))
    crop_img=crop_img.reshape((1,img_size,img_size,3))
    pre=np.argmax(infer_model.predict(crop_img),axis=1)
    predicts.append(pre[0])

In [52]:
submit=pd.DataFrame()
submit["image_id"]=ids
submit["label"]=predicts
submit.to_csv("submission.csv",index=False)

In [53]:
submit

,image_id,label
0,2216849948.jpg,4
